In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from glob import glob
import time

# 1. Load video

https://m.blog.naver.com/dldudcks1779/222064172394
resize : real-time camera capture 이미지 크기에 맞게 변환
1) 변환할 이미지
2) 변환할 이미지 크기(가로, 세로)
- interpolation : 보간법 지정
  - 보간법 : 알려진 데이터 지점 내에서 새로운 데이터 지점을 구성하는 방식
  - cv2.INTER_NEAREST : 최근방 이웃 보간법
  - cv2.INTER_LINEAR(default) : 양선형 보간법(2x2 이웃 픽셀 참조)
  - cv2.INTER_CUBIC : 3차 회선 보간법(4x4 이웃 픽셀 참조)
  - cv2.INTER_LANCZOS4 : Lanczos 보간법(8x8 이웃 픽셀 참조)
  - cv2.INTER_AREA : 픽셀 영역 관계를 이용한 resampling 방법으로 이미지 축소시 효과적
- resize_frame = cv2.resize(frame, (640, 480), interpolation=cv2.INTER_CUBIC)

In [2]:
path = './원시데이터/'
file_list = glob(path+'*')

file_list

['./원시데이터\\NIA_SL_SEN0001_REAL01_D.mp4',
 './원시데이터\\NIA_SL_SEN0001_REAL01_F.mp4',
 './원시데이터\\NIA_SL_SEN0001_REAL01_L.mp4',
 './원시데이터\\NIA_SL_SEN0001_REAL01_R.mp4',
 './원시데이터\\NIA_SL_SEN0001_REAL01_U.mp4',
 './원시데이터\\NIA_SL_SEN0002_REAL01_D.mp4',
 './원시데이터\\NIA_SL_SEN0002_REAL01_F.mp4',
 './원시데이터\\NIA_SL_SEN0002_REAL01_L.mp4',
 './원시데이터\\NIA_SL_SEN0002_REAL01_R.mp4',
 './원시데이터\\NIA_SL_SEN0002_REAL01_U.mp4',
 './원시데이터\\NIA_SL_SEN0003_REAL01_D.mp4',
 './원시데이터\\NIA_SL_SEN0003_REAL01_F.mp4',
 './원시데이터\\NIA_SL_SEN0003_REAL01_L.mp4',
 './원시데이터\\NIA_SL_SEN0003_REAL01_R.mp4',
 './원시데이터\\NIA_SL_SEN0003_REAL01_U.mp4',
 './원시데이터\\NIA_SL_SEN0004_REAL01_D.mp4',
 './원시데이터\\NIA_SL_SEN0004_REAL01_F.mp4',
 './원시데이터\\NIA_SL_SEN0004_REAL01_L.mp4',
 './원시데이터\\NIA_SL_SEN0004_REAL01_R.mp4',
 './원시데이터\\NIA_SL_SEN0004_REAL01_U.mp4',
 './원시데이터\\NIA_SL_SEN0005_REAL01_D.mp4',
 './원시데이터\\NIA_SL_SEN0005_REAL01_F.mp4',
 './원시데이터\\NIA_SL_SEN0005_REAL01_L.mp4',
 './원시데이터\\NIA_SL_SEN0005_REAL01_R.mp4',
 './원시데이터\\NIA_S

In [3]:
import cv2

# video 정보
vid = cv2.VideoCapture(file_list[1])

if not vid.isOpened(): 
    print("could not open :",fn)
    
length = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = vid.get(cv2.CAP_PROP_FPS)

print(length, width, height, fps)

117 1920 1080 30.0


In [4]:
start = time.time()
vid = cv2.VideoCapture(file_list[1])

if vid.isOpened() == False:
    print('video read error')
        
while vid.isOpened():
    
    ret, frame = vid.read()
    
    if not ret:
        break
    
    resize_frame = cv2.resize(frame, (640, 480), interpolation=cv2.INTER_CUBIC)
    cv2.imshow('frame', resize_frame)

    if cv2.waitKey(25) % 0xFF == ord('q'):
        break

            
vid.release()
cv2.destroyAllWindows()
print('time cost : %.2f' %(time.time() - start))

time cost : 5.85


In [31]:
# frame size 비교
print(frame.shape)
print(resize_frame.shape)

(1080, 1920, 3)
(480, 640, 3)


# 2. Holistic 적용 및 Keypoint 추출

In [5]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [6]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction. Model : holistic, image를 입력해 스켈레톤을 좌표값을 생성.
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

# 입력 image에 landmarks 얹는 함수.
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    
def draw_styled_landmarks(image, results):
    # image : frame, results : model.predict(image)
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

- 특이사항 : mediapipe 적용시 원본 동영상에 비해 동영상이 느려짐. (resize관계 무)

In [7]:
path = './원시데이터/'
file_list = glob(path+'*')

start = time.time()
vid = cv2.VideoCapture(file_list[1])

if vid.isOpened() == False:
    print('video read error')
       
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while vid.isOpened():

        ret, frame = vid.read()
        
        # check input ret
        if not ret:
            break

        frame = cv2.resize(frame, (640, 480), interpolation=cv2.INTER_CUBIC)
        image, results = mediapipe_detection(frame, holistic) # 예측 : 스켈레톤 생성.
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results) # 이미지에 스켈레톤 얹기.

        # Show to screen
        cv2.imshow('OpenCV Feed', image)        
        
        if cv2.waitKey(25) % 0xFF == ord('q'):
            break


    vid.release()
    cv2.destroyAllWindows()
    
print('time cost : %.2f' %(time.time() - start))

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [8]:
# pose, face, lh, rh 좌표값 리스트화.
# 132 = 33 x 4
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
# 1404 = 468 x 3
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
# 63 = 21 x 3
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

print(pose.shape, face.shape, lh.shape, rh.shape)

(132,) (1404,) (63,) (63,)


In [9]:
# 함수화 : 각 좌표값을 한 ndarray로 concat.
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [12]:
result_test = extract_keypoints(results)

# 1662 = 132 + 1404 + 63*2
result_test.shape, result_test

((1662,),
 array([ 0.50001252,  0.22932841, -0.53278667, ...,  0.38931453,
         0.96375936,  0.01161412]))

# 3. Collect Keypoint Values for Training and Testing

## 파일정리

In [10]:
import json
import collections

def call_info(file):
    with open(file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
        name = json_data['data'][0]['attributes'][0]['name'] 

        # print(json.dumps(json_data, indent=4) )
        # print(name)
        return name

In [11]:
# sample files
sample_video_path = './원시데이터'
sample_morpheme_path = './라벨링데이터/morpheme/'

sample_video_files = glob(sample_video_path + '/*')
sample_morpheme_files = glob(sample_morpheme_path + '/*')

# count sample file 
actions=[]
for video_file, morpheme_file in zip(sample_video_files, sample_morpheme_files):
    action = call_info(morpheme_file)
    actions.append(action)
    
actions = collections.Counter(actions)
actions

Counter({'왼쪽': 10,
         '오른쪽': 10,
         '여기': 10,
         '저기': 10,
         '운전': 15,
         '가다': 5,
         '시간': 10,
         '급하다': 10,
         '약속': 10,
         '나': 10,
         '당신': 10,
         '그남자': 10,
         '말하다': 5,
         '말해주다': 5,
         '위험': 10,
         '항상': 10})

In [110]:
# make dir
path = './custom_keypoints'
for video_file, morpheme_file in zip(sample_video_files, sample_morpheme_files):
    action = call_info(morpheme_file)
    file_name = video_file.split('\\')[-1][:-4]
    save_path = os.path.join(path, action, file_name)
    os.makedirs(save_path)

## Keypoint 추출

In [9]:
# set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # LOOP through actions
    for video_file, morpheme_file in zip(sample_video_files, sample_morpheme_files):
        start = time.time()

        action = call_info(morpheme_file)
        file_name = video_file.split('\\')[-1][:-4]

        
        # load video file
        print(file_name, end='\t')
        vid = cv2.VideoCapture(video_file)

        if vid.isOpened() == False:
            print('video read error')
        
        sequence_length = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
        
        for frame_num in range(sequence_length):
            ret, frame = vid.read()

            # check input ret
            if not ret:
                break

                
            frame = cv2.resize(frame, (640, 480), interpolation=cv2.INTER_CUBIC)
            image, results = mediapipe_detection(frame, holistic) # 예측 : 스켈레톤 생성.

            # Draw landmarks
            draw_styled_landmarks(image, results) # 이미지에 스켈레톤 얹기.

            # Show to screen
            cv2.imshow('OpenCV Feed', image)        
            
            # NEW Export keypoints
            save_path = os.path.join('./custom_keypoints', action, file_name, str(frame_num))
            keypoints = extract_keypoints(results) # result : 스켈레톤 좌표값.
            np.save(save_path, keypoints)
            
            if cv2.waitKey(25) % 0xFF == ord('q'):
                break

        vid.release()
        cv2.destroyAllWindows()

        print(f'time cost : {(time.time() - start):.2f}' )

NIA_SL_SEN0001_REAL01_D	time cost : 18.68
NIA_SL_SEN0001_REAL01_F	time cost : 20.18
NIA_SL_SEN0001_REAL01_L	time cost : 18.92
NIA_SL_SEN0001_REAL01_R	

KeyboardInterrupt: 